# <font color=#003091> Big Data Project: IRI Academic Data Set Analysis </font>
### <font color=#0098df> Master in Computer Science & Business Technology</font>
#### <font color=#a6a6a6> Year: 2023</font>  
**Nicholas Dieke**

This notebook shows some analyses on the dataset.

In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lpad, lit
from pyspark.sql import functions as F


## Loading Datasets
Creates a SparkSession named spark and reads multiple CSV files into PySpark DataFrames.

In [3]:
spark = SparkSession.builder.getOrCreate()
stores = spark.read.csv("Saltsnck_US_1of2/Delivery_Stores.csv", header=True)
products = spark.read.csv("Saltsnck_US_1of2/prod_saltsnck.csv", header=True)
attributes = spark.read.csv("Saltsnck_US_1of2/saltsnck_prod_attr.csv", header=True)
weeks = spark.read.csv("Saltsnck_US_1of2/IRI week translation_2008_2017.csv", header=True)
sell_out = spark.read.csv("saltsnck_groc.csv", header=True)

In [4]:
sell_out.head(3)

[Row(IRI_KEY='234212', WEEK='1635', SY='0', GE='3', VEND='78271', ITEM='2060', UNITS='6', DOLLARS='12.00', F='NONE', D='0', PR='1'),
 Row(IRI_KEY='234212', WEEK='1635', SY='0', GE='3', VEND='78271', ITEM='2061', UNITS='4', DOLLARS='8.79', F='NONE', D='0', PR='1'),
 Row(IRI_KEY='234212', WEEK='1635', SY='0', GE='1', VEND='84114', ITEM='11336', UNITS='5', DOLLARS='15.00', F='NONE', D='0', PR='0')]

## Create UPC Feature
Perform column transformations (padding) to create UPC feature

In [5]:
attributes = attributes.withColumn("SY", lpad(col("SY").cast("string"), 2, "0"))
attributes = attributes.withColumn("GE", lpad(col("GE").cast("string"), 2, "0"))
attributes = attributes.withColumn("VEND", lpad(col("VEND").cast("string"), 5, "0"))
attributes = attributes.withColumn("ITEM", lpad(col("ITEM").cast("string"), 5, "0"))

attributes = attributes.withColumn("UPC", concat(col("SY"), lit("-"), col("GE"), lit("-"), col("VEND"), lit("-"), col("ITEM")))

In [6]:
sell_out = sell_out.withColumn("SY", lpad(col("SY").cast("string"), 2, "0"))
sell_out = sell_out.withColumn("GE", lpad(col("GE").cast("string"), 2, "0"))
sell_out = sell_out.withColumn("VEND", lpad(col("VEND").cast("string"), 5, "0"))
sell_out = sell_out.withColumn("ITEM", lpad(col("ITEM").cast("string"), 5, "0"))

sell_out = sell_out.withColumn("UPC", concat(col("SY"), lit("-"), col("GE"), lit("-"), col("VEND"), lit("-"), col("ITEM")))

## Merging Datasets
We merge the five datasets together using either IRI_KEY or UPC

In [7]:
merged_df = sell_out.join(stores, on="IRI_KEY", how="left")


In [8]:
weeks = weeks.select("IRI Week", "Calendar week starting on", "Calendar week ending on")
merged_df = merged_df.join(weeks, merged_df["WEEK"] == weeks["IRI Week"], "left")

In [9]:
# Select relevant features and merge
products = products.select('UPC', 'ITEM', 'PRODUCT TYPE', 'L3', 'L5', 'L9')
merged_df = merged_df.join(products, on="UPC", how="left")

In [10]:
# Select relevant features and merge
attributes = attributes.select('UPC', 'PRODUCT TYPE')
merged_df = merged_df.join(attributes, on='UPC', how='left')

In [11]:
merged_df.head(3)

[Row(UPC='00-03-78271-02060', IRI_KEY='234212', WEEK='1635', SY='00', GE='03', VEND='78271', ITEM='02060', UNITS='6', DOLLARS='12.00', F='NONE', D='0', PR='1', _c0='239', OU='GR', EST_ACV='34.041990000000006', Market_Name='CHICAGO', Open='435', Clsd='9998', MskdName='Chain42', IRI Week='1635', Calendar week starting on='2010-12-27', Calendar week ending on='2011-01-02', ITEM=None, PRODUCT TYPE=None, L3=None, L5=None, L9=None, PRODUCT TYPE='POTATO CHIP'),
 Row(UPC='00-03-78271-02061', IRI_KEY='234212', WEEK='1635', SY='00', GE='03', VEND='78271', ITEM='02061', UNITS='4', DOLLARS='8.79', F='NONE', D='0', PR='1', _c0='239', OU='GR', EST_ACV='34.041990000000006', Market_Name='CHICAGO', Open='435', Clsd='9998', MskdName='Chain42', IRI Week='1635', Calendar week starting on='2010-12-27', Calendar week ending on='2011-01-02', ITEM=None, PRODUCT TYPE=None, L3=None, L5=None, L9=None, PRODUCT TYPE='POTATO CHIP'),
 Row(UPC='00-01-84114-11336', IRI_KEY='234212', WEEK='1635', SY='00', GE='01', VEND

## Pepsico Revenue and Volume for 2011

- 2011 Pepsico Revenue: **$332,054,362**

- 2011 Pepsico Units Sold: **126,315,847 units**

In [12]:
merged_df.filter(merged_df["L3"].contains("PEPS")).groupBy('VEND').agg(F.first("L3")).head(10)

[Row(VEND='28400', first(L3)='PEPSICO INC'),
 Row(VEND='30000', first(L3)='PEPSICO INC'),
 Row(VEND='71461', first(L3)='PEPSICO INC')]

In [15]:
pepsico_vend_codes = ['28400', '30000', '71461']

merged_df_2011 = merged_df.filter(merged_df["Calendar week starting on"].contains("2011"))
pepsico_2011 = merged_df_2011.filter(merged_df_2011["VEND"].isin(pepsico_vend_codes))

merged_df_2012 = merged_df.filter(merged_df["Calendar week starting on"].contains("2012"))
pepsico_2012 = merged_df_2012.filter(merged_df_2012["VEND"].isin(pepsico_vend_codes))

sum_dollars_2011 = pepsico_2011.agg(F.sum("DOLLARS")).collect()[0][0]
sum_units_2011 = pepsico_2011.agg(F.sum("UNITS")).collect()[0][0]
print("2011 Revenue: ",sum_dollars_2011)
print("2011 Units: ",sum_units_2011)

2011 Revenue:  332054362.18951464
2011 Units:  126315847.0


## Pepsico Top 3 Regions 2011

1. **Los Angeles** ($26,212,674)
2. **New York** ($21,566,451)
3. **Chicago** ($14,315,358)

In [16]:
sum__2011_df = pepsico_2011.groupBy("Market_Name").agg(F.sum("DOLLARS").alias("SUM_DOLLARS")).orderBy(F.desc("SUM_DOLLARS"))
sum__2011_df.head(3)

[Row(Market_Name='LOS ANGELES', SUM_DOLLARS=26212674.080007784),
 Row(Market_Name='NEW YORK', SUM_DOLLARS=21566451.110004514),
 Row(Market_Name='CHICAGO', SUM_DOLLARS=14315358.299999913)]

## Pepsico Top 3 Regions 2012

1. **Los Angeles** ($27,206,073)
2. **New York** ($22,033,104)
3. **Chicago** ($14,271,019)

In [17]:
sum__2012_df = pepsico_2012.groupBy("Market_Name").agg(F.sum("DOLLARS").alias("SUM_DOLLARS")).orderBy(F.desc("SUM_DOLLARS"))
sum__2012_df.head(3)

[Row(Market_Name='LOS ANGELES', SUM_DOLLARS=27206073.270008586),
 Row(Market_Name='NEW YORK', SUM_DOLLARS=22033103.980004217),
 Row(Market_Name='CHICAGO', SUM_DOLLARS=14271018.740001548)]

## Pepsico Market Share 2011

2011 Market Share: **59.13%**

In [19]:
market_share_2011_pepsico = pepsico_2011.agg(F.sum("DOLLARS")).collect()[0][0] / merged_df_2011.agg(F.sum("DOLLARS")).collect()[0][0]
market_share_2011_pepsico

0.5913256213344841

## Pepsico Market Share 2012

2012 Market Share: **58.83%**

In [20]:
market_share_2012_pepsico = pepsico_2012.agg(F.sum("DOLLARS")).collect()[0][0] / merged_df_2012.agg(F.sum("DOLLARS")).collect()[0][0]
market_share_2012_pepsico

0.5883610351197499

## Top PepsiCo Customers in New York in 2011

1. **Chain98** ($4,815,088)
2. **Chain112** ($4,725,130)
3. **Chain5** ($4,574,273)
4. **Chain73** ($2,979,815)
5. **Chain110** ($1,111,449)

In [21]:
pepsico_2011_NY = pepsico_2011.filter(merged_df["Market_Name"].contains("NEW YORK"))
retailer_revenues_2011 = pepsico_2011_NY.groupBy("MskdName").agg(F.sum("DOLLARS").alias("SUM_DOLLARS")).orderBy(F.desc("SUM_DOLLARS"))
retailer_revenues_2011.head(5)

## Top PepsiCo Customers in New York in 2012

1. **Chain112** ($4,835,976)
2. **Chain98** ($4,791,504)
3. **Chain5** ($4,757,187)
4. **Chain73** ($3,017,525)
5. **Chain110** ($1,141,277)

In [24]:
pepsico_2012_NY = pepsico_2012.filter(merged_df["Market_Name"].contains("NEW YORK"))
retailer_revenues_2012 = pepsico_2012_NY.groupBy("MskdName").agg(F.sum("DOLLARS").alias("SUM_DOLLARS")).orderBy(F.desc("SUM_DOLLARS"))
retailer_revenues_2012.head(5)

[Row(MskdName='Chain112', SUM_DOLLARS=4835975.539999725),
 Row(MskdName='Chain98', SUM_DOLLARS=4791504.039999706),
 Row(MskdName='Chain5', SUM_DOLLARS=4757187.459999587),
 Row(MskdName='Chain73', SUM_DOLLARS=3017525.3799998136),
 Row(MskdName='Chain110', SUM_DOLLARS=1141277.0799999998)]

## Top Products in New York in 2011

1. **00-03-28400-03345** ($1,022,581)
2. **00-01-28400-03875** ($1,015,517)
3. **00-02-28400-06408** ($975,795)
4. **00-02-28400-06399** ($830,449)
5. **00-01-28400-03346** ($608,710)

In [27]:
merged_df_2011_NY = merged_df_2011.filter(merged_df["Market_Name"].contains("NEW YORK"))
top_products_dollars_2011 = merged_df_2011_NY.groupBy("UPC").agg(F.sum("DOLLARS").alias("SUM_DOLLARS"), F.avg("PR").alias("AVG_PR")).orderBy(F.desc("SUM_DOLLARS"))
top_products_dollars_2011.select("UPC", "SUM_DOLLARS").head(5)

[Row(UPC='00-03-28400-03345', SUM_DOLLARS=1022580.7299999997),
 Row(UPC='00-01-28400-03875', SUM_DOLLARS=1015516.7400000006),
 Row(UPC='00-02-28400-06408', SUM_DOLLARS=975795.2800000004),
 Row(UPC='00-02-28400-06399', SUM_DOLLARS=830449.3699999999),
 Row(UPC='00-01-28400-03346', SUM_DOLLARS=608710.1100000002)]

## Top Products in New York in 2012

1. **00-01-28400-03875** ($1,399,523)
2. **00-03-28400-03345** ($1,382,586)
3. **00-03-28400-06408** ($1,112,857)
4. **00-01-28400-03854** ($753,409)
5. **00-02-28400-06399** ($741,246)

In [28]:
merged_df_2012_NY = merged_df_2012.filter(merged_df["Market_Name"].contains("NEW YORK"))
top_products_dollars_2012 = merged_df_2012_NY.groupBy("UPC").agg(F.sum("DOLLARS").alias("SUM_DOLLARS"), F.avg("PR").alias("AVG_PR")).orderBy(F.desc("SUM_DOLLARS"))
top_products_dollars_2012.select("UPC", "SUM_DOLLARS").head(5)

[Row(UPC='00-01-28400-03875', SUM_DOLLARS=1399523.4899999995),
 Row(UPC='00-03-28400-03345', SUM_DOLLARS=1382585.5199999996),
 Row(UPC='00-03-28400-06408', SUM_DOLLARS=1112857.14),
 Row(UPC='00-01-28400-03854', SUM_DOLLARS=753409.0200000005),
 Row(UPC='00-02-28400-06399', SUM_DOLLARS=741246.2599999997)]

## Promotional Strength of Top 3 Products in New York in 2011

1. 00-01-28400-03875 (**0.542**)
2. 00-03-28400-03345 (**0.545**)
3. 00-03-28400-06408 (**0.466**)

In [30]:
top_products_dollars_2011.select("UPC", "AVG_PR").head(3)

[Row(UPC='00-03-28400-03345', AVG_PR=0.5421597633136095),
 Row(UPC='00-01-28400-03875', AVG_PR=0.545025931164545),
 Row(UPC='00-02-28400-06408', AVG_PR=0.46569400630914826)]

## Promotional Strength of Top 3 Products in New York in 2012

1. 00-01-28400-03875 (**0.586**)
2. 00-03-28400-03345 (**0.608**)
3. 00-03-28400-06408 (**0.478**)

In [31]:
top_products_dollars_2012.select("UPC", "AVG_PR").head(3)

[Row(UPC='00-01-28400-03875', AVG_PR=0.5857142857142857),
 Row(UPC='00-03-28400-03345', AVG_PR=0.6079986209274263),
 Row(UPC='00-03-28400-06408', AVG_PR=0.4783657990001724)]

## Pepsico Volume Sold on Deal (VSoD) in 2011

VSoD: **3,956,230 units (50.53%)**

In [35]:
vsod_ny_2011 = pepsico_2011_NY.filter(pepsico_2011_NY["PR"] != 0).agg(F.sum("UNITS")).collect()[0][0]
vsod_pct_ny_2011 = vsod_ny_2011/(merged_df_2011_NY.filter(pepsico_2011_NY["PR"]!= 0).agg(F.sum("UNITS")).collect()[0][0])
print("VSoD: ",vsod_ny_2011)
print("% VSoD: ",vsod_pct_ny_2011)

VSoD:  3956230.0
% VSoD:  0.5053279235563515


## Pepsico Volume Sold on Deal (VSoD) in 2012

VSoD: **4,255,249 units (53.94%)**

In [36]:
vsod_ny_2012 = pepsico_2012_NY.filter(pepsico_2012_NY["PR"] != 0).agg(F.sum("UNITS")).collect()[0][0]
vsod_pct_ny_2012 = vsod_ny_2012/(merged_df_2012_NY.filter(pepsico_2012_NY["PR"] != 0).agg(F.sum("UNITS")).collect()[0][0])
print("VSoD: ",vsod_ny_2012)
print("% VSoD: ",vsod_pct_ny_2012)

VSoD:  4255249.0
% VSoD:  0.5393542020141641


## Time Series Chart Data

In [39]:
merged_df_NY = merged_df.filter(merged_df["Market_Name"].contains("NEW YORK"))

chart_df = merged_df_NY.groupBy("UPC","WEEK").agg(F.sum("DOLLARS"), F.sum("UNITS"), F.avg("PR"), F.avg("D"), F.first("VEND"), F.first("L3"), F.first("L5"), F.first("L9"))
chart_df.head(5)

[Row(UPC='00-01-00901-00003', WEEK='1643', sum(DOLLARS)=23.87, sum(UNITS)=13.0, avg(PR)=0.0, avg(D)=0.3333333333333333, first(VEND)='00901', first(L3)='RAB HOLDINGS', first(L5)='GUILTLESS GOURMET', first(L9)='+GLTLG MHNCH TRCHP TRNGL 7OZ'),
 Row(UPC='00-01-00901-00003', WEEK='1656', sum(DOLLARS)=19.369999999999997, sum(UNITS)=13.0, avg(PR)=1.0, avg(D)=0.0, first(VEND)='00901', first(L3)='RAB HOLDINGS', first(L5)='GUILTLESS GOURMET', first(L9)='+GLTLG MHNCH TRCHP TRNGL 7OZ'),
 Row(UPC='00-01-00901-00003', WEEK='1668', sum(DOLLARS)=14.95, sum(UNITS)=5.0, avg(PR)=0.0, avg(D)=0.0, first(VEND)='00901', first(L3)='RAB HOLDINGS', first(L5)='GUILTLESS GOURMET', first(L9)='+GLTLG MHNCH TRCHP TRNGL 7OZ'),
 Row(UPC='00-01-00901-00003', WEEK='1675', sum(DOLLARS)=13.93, sum(UNITS)=7.0, avg(PR)=1.0, avg(D)=0.0, first(VEND)='00901', first(L3)='RAB HOLDINGS', first(L5)='GUILTLESS GOURMET', first(L9)='+GLTLG MHNCH TRCHP TRNGL 7OZ'),
 Row(UPC='00-01-00901-00003', WEEK='1676', sum(DOLLARS)=5.98, sum(UNIT

In [40]:
chart_df.toPandas().to_csv("chart_data.csv")